In [76]:
import numpy as np
import pandas as pd

In [77]:
df = pd.read_csv('database_fires.csv') 

In [78]:
df.head()

,id,estado,estacao,data,precipitacao,temp_max,temp_min,insolacao,evaporacao_piche,temp_comp_med,umidade_rel_med,vel_vento_med,altitude,fires
0,1,AC,50484,01/01/2016,NaN,31.4,NaN,2.3,NaN,NaN,96.0,0.00000,170.0,0
1,2,AC,50484,02/01/2016,0.0,32.5,23.3,4.4,NaN,26.44,95.0,0.51444,170.0,0
2,3,AC,50484,03/01/2016,35.2,30.5,24.0,0.6,NaN,25.78,97.0,0.51444,170.0,0
3,4,AC,50484,04/01/2016,60.2,31.7,22.3,2.0,NaN,25.94,96.0,0.00000,170.0,0
4,5,AC,50484,05/01/2016,28.4,28.5,23.0,0.1,NaN,25.04,95.0,0.00000,170.0,0


In [79]:
# Removendo coletas com menos de 11 valores
df.dropna(thresh=11, inplace=True)

In [80]:
#porcentagem de valores faltantes
(df.isna().sum() / df.count()) * 100 

id                   0.000000
estado               0.000000
estacao              0.000000
data                 0.000000
precipitacao         0.136133
temp_max             6.013400
temp_min             2.029289
insolacao           16.107691
evaporacao_piche    36.627985
temp_comp_med       11.842538
umidade_rel_med     10.936761
vel_vento_med       16.124355
altitude             0.000000
fires                0.000000
dtype: float64

In [81]:
#removendo linhas sem valores de precipitação
df.dropna(subset=['precipitacao'], inplace=True) 

In [82]:
df.corr()['fires'].sort_values()

umidade_rel_med    -0.357786
precipitacao       -0.169521
temp_min           -0.032186
estacao             0.002006
altitude            0.015150
vel_vento_med       0.034038
id                  0.039549
temp_comp_med       0.112953
temp_max            0.229382
evaporacao_piche    0.271760
insolacao           0.321320
fires               1.000000
Name: fires, dtype: float64

In [83]:
#trocar estados por região
def getRegiao(estado):
    if (estado in ['AM', 'RR', 'AC', 'RO', 'PA', 'TO', 'AP']):
        return 1
    elif (estado in ['BA', 'SE', 'AL', 'PE', 'PB', 'RN', 'CE', 'MA', 'PI']):
        return 2
    elif (estado in ['GO', 'MT', 'MS', 'DF']):
        return 3        
    elif (estado in ['PR', 'SC', 'RS']):
        return 4
    elif (estado in ['ES', 'MG', 'SP', 'RJ']):
        return 5
df['regiao'] = df['estado'].map(getRegiao)

In [84]:
#trocar as datas completas pelo mês apenas
import re
df['mes'] = df['data'].map(lambda x: int(re.search('/(.+?)/', x).group(1)))

In [85]:
#remover colunas de estado e data
df.drop(['estado', 'data', 'id', 'altitude'], axis=1, inplace=True)
df.head()

,estacao,precipitacao,temp_max,temp_min,insolacao,evaporacao_piche,temp_comp_med,umidade_rel_med,vel_vento_med,fires,regiao,mes
1,50484,0.0,32.5,23.3,4.4,NaN,26.44,95.0,0.51444,0,1,1
2,50484,35.2,30.5,24.0,0.6,NaN,25.78,97.0,0.51444,0,1,1
3,50484,60.2,31.7,22.3,2.0,NaN,25.94,96.0,0.00000,0,1,1
4,50484,28.4,28.5,23.0,0.1,NaN,25.04,95.0,0.00000,0,1,1
5,50484,2.7,31.7,23.3,1.2,NaN,26.28,95.0,0.00000,0,1,1


In [86]:
#Importando biblioteca para completar os dados faltantes
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [87]:
#Completando dados faltantes pelo IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(df)

df = pd.DataFrame(imp.transform(df), columns=df.columns)

In [91]:
from sklearn.preprocessing import OneHotEncoder
def oneHot(column):
    cat_encoder = OneHotEncoder()
    resultado = cat_encoder.fit_transform(column)
    resultado = pd.DataFrame(resultado.toarray())
    resultado.columns = resultado.columns.map(lambda x: column.columns[0]+str(x))
    return resultado

df = pd.concat([df, oneHot(df[['regiao']]), oneHot(df[['mes']])], axis=1, sort=False)
df.drop(['regiao', 'mes'], axis=1, inplace=True)
df.head()

,estacao,precipitacao,temp_max,temp_min,insolacao,evaporacao_piche,temp_comp_med,umidade_rel_med,vel_vento_med,fires,...,mes2,mes3,mes4,mes5,mes6,mes7,mes8,mes9,mes10,mes11
0,50484.0,0.0,32.5,23.3,4.4,1.366810,26.44,95.0,0.51444,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50484.0,35.2,30.5,24.0,0.6,0.861250,25.78,97.0,0.51444,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50484.0,60.2,31.7,22.3,2.0,0.074433,25.94,96.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,50484.0,28.4,28.5,23.0,0.1,0.402582,25.04,95.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50484.0,2.7,31.7,23.3,1.2,1.191513,26.28,95.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
df.columns

Index(['estacao', 'precipitacao', 'temp_max', 'temp_min', 'insolacao',
       'evaporacao_piche', 'temp_comp_med', 'umidade_rel_med', 'vel_vento_med',
       'fires', 'regiao0', 'regiao1', 'regiao2', 'regiao3', 'regiao4', 'mes0',
       'mes1', 'mes2', 'mes3', 'mes4', 'mes5', 'mes6', 'mes7', 'mes8', 'mes9',
       'mes10', 'mes11'],
      dtype='object')

In [93]:
target = df['fires'].copy()
df.drop('fires', axis=1, inplace=True)

In [94]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,-0.18182,-0.363162,0.367913,0.799908,-0.713199,-1.012397,0.445028,1.563680,-0.969392,2.147155,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
1,-0.18182,2.812267,-0.095687,0.968330,-1.868592,-1.171825,0.273634,1.707139,-0.969392,2.147155,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
2,-0.18182,5.067543,0.182473,0.559305,-1.442921,-1.419946,0.315184,1.635409,-1.383517,2.147155,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
3,-0.18182,2.198832,-0.559287,0.727727,-2.020618,-1.316465,0.081465,1.563680,-1.383517,2.147155,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
4,-0.18182,-0.119592,0.182473,0.799908,-1.686162,-1.067677,0.403478,1.563680,-1.383517,2.147155,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108


In [95]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=42)
X_train

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
12701,0.352161,-0.363162,0.530173,0.727727,0.350978,-0.448924,0.767041,0.667060,-0.555267,2.147155,...,-0.333404,-0.325661,-0.322849,3.488946,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
7060,0.974887,-0.345120,-0.536107,0.775847,0.928675,0.038720,0.294409,1.348491,-0.578515,-0.465733,...,-0.333404,-0.325661,3.097428,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
97903,-0.368657,1.783861,-1.324226,-0.234685,-1.777377,-0.844256,-1.040387,0.900181,0.226486,-0.465733,...,-0.333404,3.070673,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
155646,-1.390730,-0.363162,0.344733,0.270581,1.141510,3.129135,0.502160,-1.144112,1.272988,-0.465733,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,3.477010,-0.280364,-0.283109,-0.278171,-0.280108
164218,-1.520339,-0.363162,0.599713,0.607425,0.411788,-0.510254,0.678747,1.061572,0.763154,-0.465733,...,-0.333404,3.070673,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1.188565,0.142020,0.112933,0.107048,0.138143,-1.065000,0.030510,0.971910,-0.041847,-0.465733,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
103694,-1.132407,-0.236866,-1.022887,-1.076796,1.111105,-0.907326,-1.196199,0.326344,0.763154,-0.465733,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
131932,0.028040,2.018410,0.391093,0.511184,-0.460814,-1.001931,0.595647,0.577398,-0.846849,2.147155,...,-0.333404,-0.325661,3.097428,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108
146867,0.812943,-0.101550,0.112933,1.232993,0.807054,-0.528908,0.969598,0.200817,0.199653,-0.465733,...,-0.333404,-0.325661,-0.322849,-0.286619,-0.282316,-0.287603,-0.280364,-0.283109,-0.278171,-0.280108


In [96]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(3)
knn.fit(X_train, y_train)
knn.score(X_train, y_train)

0.8839243801476551

In [97]:
knn.score(X_test, y_test)

0.793112277557291

In [98]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=14, min_samples_split=14, random_state=42)
print('treinando...')
tree.fit(X_train, y_train)
print('testando...')
tree.score(X_train, y_train)

treinando...
testando...


0.835461741987795

In [99]:
tree.score(X_test, y_test)

0.7959714931933598

In [100]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(max_depth=20, min_samples_split=5, random_state=42)
print('treinando...')
forest.fit(X_train, y_train)
print('testando...')
forest.score(X_train, y_train)

treinando...
testando...


0.9080410105406905

In [101]:
forest.score(X_test, y_test)

0.8193573166047882

In [53]:
gridPar = {'max_depth' : np.arange(5, 21, 3), 'min_samples_split' : np.arange(2, 15, 3)}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(forest, gridPar, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=20,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=10,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                   

In [56]:
grid_search.best_params_

{'max_depth': 20, 'min_samples_split': 5}

In [89]:
respostas = pd.read_csv('respostas.csv')['id']
respId = respostas['id']
respostas.head()

,id,estado,estacao,data,precipitacao,temp_max,temp_min,insolacao,evaporacao_piche,temp_comp_med,umidade_rel_med,vel_vento_med,altitude
0,251768,AC,50484,27/01/2016,5.8,34.7,23.7,8.8,NaN,27.84,92.00,1.028880,170.0
1,251769,AC,50484,28/01/2016,0.0,30.0,23.7,0.2,NaN,26.60,93.00,0.000000,170.0
2,251770,AC,50484,29/01/2016,0.0,35.5,24.1,7.1,NaN,27.78,97.00,0.000000,170.0
3,251771,AC,50484,05/08/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,170.0
4,251772,AC,88948,28/05/2019,0.0,34.7,21.6,8.3,2.0,26.56,86.75,0.666667,160.0


In [90]:
respostas['mes'] = respostas['data'].map(lambda x: int(re.search('/(.+?)/', x).group(1)))
respostas['regiao'] = respostas['estado'].map(getRegiao)
respostas.drop(['estado', 'data', 'id', 'altitude'], axis=1, inplace=True)
imp.fit(df.drop('fires', axis=1))

respostas = pd.DataFrame(imp.transform(respostas), columns=respostas.columns)
respostas.head()

,estacao,precipitacao,temp_max,temp_min,insolacao,evaporacao_piche,temp_comp_med,umidade_rel_med,vel_vento_med,mes,regiao
0,50484.0,5.80000,34.700000,23.700000,8.800000,1.826289,27.84000,92.000000,1.028880,1.0,1.0
1,50484.0,0.00000,30.000000,23.700000,0.200000,1.045803,26.60000,93.000000,0.000000,1.0,1.0
2,50484.0,0.00000,35.500000,24.100000,7.100000,1.115198,27.78000,97.000000,0.000000,1.0,1.0
3,50484.0,4.23681,31.987726,19.703147,5.941907,3.062923,24.44681,70.916977,0.000000,8.0,1.0
4,88948.0,0.00000,34.700000,21.600000,8.300000,2.000000,26.56000,86.750000,0.666667,5.0,1.0


In [102]:
respostas = pd.concat([respostas, oneHot(respostas[['regiao']]), oneHot(respostas[['mes']])], axis=1, sort=False)
respostas.drop(['regiao', 'mes'], axis=1, inplace=True)

respostas = pd.DataFrame(scaler.fit_transform(respostas))
respostas.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,-0.182251,0.334690,0.760394,0.954934,0.548631,-0.933214,0.829348,1.503579,-0.490351,2.16076,...,-0.120438,-0.129197,-0.221008,-0.370258,-0.382752,-0.396460,-0.381279,-0.389308,-0.383255,-0.387539
1,-0.182251,-0.313401,-0.240916,0.954934,-2.172974,-1.232524,0.526712,1.572377,-1.316540,2.16076,...,-0.120438,-0.129197,-0.221008,-0.370258,-0.382752,-0.396460,-0.381279,-0.389308,-0.383255,-0.387539
2,-0.182251,-0.313401,0.930829,1.044065,0.010639,-1.205911,0.814705,1.847569,-1.316540,2.16076,...,-0.120438,-0.129197,-0.221008,-0.370258,-0.382752,-0.396460,-0.381279,-0.389308,-0.383255,-0.387539
3,-0.182251,0.160019,0.182559,0.064319,-0.355858,-0.458976,0.001200,0.053108,-1.316540,2.16076,...,-0.120438,-0.129197,-0.221008,-0.370258,-0.382752,2.522323,-0.381279,-0.389308,-0.383255,-0.387539
4,1.323337,-0.313401,0.760394,0.486993,0.390398,-0.866597,0.516949,1.142389,-0.781207,2.16076,...,-0.120438,-0.129197,4.524724,-0.370258,-0.382752,-0.396460,-0.381279,-0.389308,-0.383255,-0.387539


In [104]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
count,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,...,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05,2.343300e+05
mean,1.888402e-14,-2.344970e-14,2.294379e-14,1.740186e-14,1.040315e-15,-6.309884e-17,1.050633e-15,-5.315739e-15,-1.126382e-14,2.159685e-13,...,1.931030e-15,-2.163994e-15,-1.848857e-15,9.836290e-15,-7.984511e-15,4.838073e-15,-2.248871e-15,-1.962972e-15,8.516764e-15,1.080695e-15
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.756768e+00,-3.631620e-01,-6.609265e+00,-6.490364e+00,-3.417444e+00,-2.457299e+00,-7.158634e+00,-4.891984e+00,-1.811192e+00,-4.657325e-01,...,-3.334045e-01,-3.256615e-01,-3.228485e-01,-2.866195e-01,-2.823164e-01,-2.876034e-01,-2.803644e-01,-2.831086e-01,-2.781714e-01,-2.801076e-01
25%,-8.582282e-01,-3.631620e-01,-4.652784e-01,-4.993486e-01,-7.036764e-01,-7.040612e-01,-4.482982e-01,-6.240730e-01,-6.933084e-01,-4.657325e-01,...,-3.334045e-01,-3.256615e-01,-3.228485e-01,-2.866195e-01,-2.823164e-01,-2.876034e-01,-2.803644e-01,-2.831086e-01,-2.781714e-01,-2.801076e-01
50%,1.635333e-02,-3.631620e-01,1.592932e-01,2.224604e-01,1.989529e-01,-2.152514e-01,2.113092e-01,1.470202e-01,-1.713130e-01,-4.657325e-01,...,-3.334045e-01,-3.256615e-01,-3.228485e-01,-2.866195e-01,-2.823164e-01,-2.876034e-01,-2.803644e-01,-2.831086e-01,-2.781714e-01,-2.801076e-01
75%,8.277079e-01,-2.098032e-01,6.692531e-01,7.277268e-01,8.374594e-01,4.486728e-01,6.735536e-01,7.567217e-01,4.948196e-01,-4.657325e-01,...,-3.334045e-01,-3.256615e-01,-3.228485e-01,-2.866195e-01,-2.823164e-01,-2.876034e-01,-2.803644e-01,-2.831086e-01,-2.781714e-01,-2.801076e-01
max,1.739727e+00,2.078231e+01,3.010432e+00,2.604430e+00,2.479333e+00,7.638620e+00,2.678344e+00,2.400101e+00,1.104023e+01,2.147155e+00,...,2.999360e+00,3.070673e+00,3.097428e+00,3.488946e+00,3.542125e+00,3.477010e+00,3.566786e+00,3.532214e+00,3.594906e+00,3.570057e+00


In [105]:
respostas.describe()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
count,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,...,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04,4.434200e+04
mean,3.141384e-16,4.110141e-15,9.899665e-16,-1.705320e-17,5.568997e-15,3.023306e-18,-5.009399e-16,-1.887835e-15,-1.333129e-15,-6.430242e-14,...,-1.465796e-16,2.975933e-15,5.157533e-15,-1.890882e-15,-2.287224e-15,-8.647731e-16,1.655495e-17,6.076256e-16,1.672621e-16,7.148973e-16
std,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,...,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00,1.000011e+00
min,-1.764715e+00,-1.475219e+00,-5.311376e+00,-5.195145e+00,-3.208863e+00,-3.917922e+00,-5.818896e+00,-4.550649e+00,-2.230463e+00,-4.628002e-01,...,-1.204382e-01,-1.291971e-01,-2.210080e-01,-3.702580e-01,-3.827520e-01,-3.964600e-01,-3.812791e-01,-3.893081e-01,-3.832553e-01,-3.875394e-01
25%,-8.449382e-01,-3.134010e-01,-4.539602e-01,-5.256105e-01,-6.539388e-01,-7.335684e-01,-3.909608e-01,-6.463599e-01,-7.036480e-01,-4.628002e-01,...,-1.204382e-01,-1.291971e-01,-2.210080e-01,-3.702580e-01,-3.827520e-01,-3.964600e-01,-3.812791e-01,-3.893081e-01,-3.832553e-01,-3.875394e-01
50%,1.686838e-02,-3.134010e-01,1.638690e-01,1.750326e-01,2.005186e-01,-1.562878e-01,1.264503e-01,9.321905e-02,-2.458752e-01,-4.628002e-01,...,-1.204382e-01,-1.291971e-01,-2.210080e-01,-3.702580e-01,-3.827520e-01,-3.964600e-01,-3.812791e-01,-3.893081e-01,-3.832553e-01,-3.875394e-01
75%,8.272805e-01,-2.687051e-01,6.538715e-01,7.543878e-01,8.018034e-01,5.871498e-01,6.926738e-01,7.468005e-01,5.035910e-01,-4.628002e-01,...,-1.204382e-01,-1.291971e-01,-2.210080e-01,-3.702580e-01,-3.827520e-01,-3.964600e-01,-3.812791e-01,-3.893081e-01,-3.832553e-01,-3.875394e-01
max,1.748467e+00,2.006793e+01,2.809522e+00,2.624144e+00,2.461713e+00,6.496434e+00,2.484088e+00,2.159553e+00,7.114948e+00,2.160760e+00,...,8.303015e+00,7.740112e+00,4.524724e+00,2.700819e+00,2.612658e+00,2.522323e+00,2.622750e+00,2.568660e+00,2.609227e+00,2.580383e+00


In [113]:
gabarito = pd.Series(forest.predict(respostas), index=respId, dtype='int32')
gabarito.head()

id
251768    0
251769    0
251770    0
251771    0
251772    0
dtype: int32

In [115]:
gabarito.to_csv('gabarito2.csv', header=False)